In [1]:
import uuid
from datetime import datetime, timedelta

class Ticket:
    def __init__(self, event_name, seat_number, event_date, owner_address, resale_allowed=True, resale_price_cap=None):
        self.ticket_id = str(uuid.uuid4())
        self.event_name = event_name
        self.seat_number = seat_number
        self.event_date = event_date
        self.owner = owner_address
        self.is_valid = True
        self.resale_allowed = resale_allowed
        self.resale_price_cap = resale_price_cap
        self.history = [(datetime.now(), "Issued", owner_address)]

    def transfer_ownership(self, new_owner, price):
        if not self.resale_allowed:
            raise PermissionError("Resale not allowed for this ticket.")
        if self.resale_price_cap is not None and price > self.resale_price_cap:
            raise ValueError("Resale price exceeds allowed cap.")
        self.owner = new_owner
        self.history.append((datetime.now(), "Transferred", new_owner))

    def validate_entry(self):
        if not self.is_valid:
            return "Invalid ticket"
        if datetime.now() > self.event_date:
            return "Event expired"
        return f"Entry Granted to {self.owner} for {self.event_name}"

class TicketSystem:
    def __init__(self):
        self.tickets = {}
        self.organizers = set()

    def add_organizer(self, address):
        self.organizers.add(address)

    def mint_ticket(self, organizer_address, event_name, seat_number, event_date, buyer_address, resale_allowed=True, resale_price_cap=None):
        if organizer_address not in self.organizers:
            raise PermissionError("Unauthorized: Only organizers can mint tickets.")
        ticket = Ticket(event_name, seat_number, event_date, buyer_address, resale_allowed, resale_price_cap)
        self.tickets[ticket.ticket_id] = ticket
        return ticket.ticket_id

    def transfer_ticket(self, ticket_id, new_owner, price):
        if ticket_id in self.tickets:
            self.tickets[ticket_id].transfer_ownership(new_owner, price)
        else:
            raise ValueError("Ticket not found.")

    def validate_ticket(self, ticket_id):
        if ticket_id in self.tickets:
            return self.tickets[ticket_id].validate_entry()
        else:
            return "Ticket ID not found."

# Example usage
if __name__ == "__main__":
    system = TicketSystem()

    # Setup organizer and mint ticket
    organizer = "0xORG123"
    buyer = "0xBUYER456"
    system.add_organizer(organizer)
    ticket_id = system.mint_ticket(
        organizer_address=organizer,
        event_name="ConcertX",
        seat_number="A12",
        event_date=datetime.now() + timedelta(days=2),
        buyer_address=buyer,
        resale_allowed=True,
        resale_price_cap=100
    )

    print(f"Ticket issued with ID: {ticket_id}")

    # Transfer ownership
    new_owner = "0xNEW789"
    system.transfer_ticket(ticket_id, new_owner, price=90)

    # Validate ticket
    print(system.validate_ticket(ticket_id))


Ticket issued with ID: 62606933-5641-4816-a035-4a98ef77845a
Entry Granted to 0xNEW789 for ConcertX
